In [1]:
import numpy as np
np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
# import Determinant class and PFHamiltonianGenerator class
from helper_PFCI import PFHamiltonianGenerator, Determinant

Set up molecule and psi4 options

In [2]:
# water
#mol_str = """
#O
#H 1 1.1
#H 1 1.1 2 104
#symmetry c1
#"""

# suppress printing of psi4 output
psi4.core.set_output_file('output.dat', False)

# LiH
mol_str = """
Li
H 1 1.0
symmetry c1
"""
options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10,
                  }


Set up the PF parameters and build the PFHamiltonian matrix. 
Note that there is now an optional keyword 'ci_level' which can be either 'cis' or 'fci'.

In [9]:
# photon energy
omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
#omega_val = 0.0

# lambda vector
lambda_vector = np.array([.01, .01, .01])
#lambda_vector = np.array([0, 0, 0.0])

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, ignore_coupling=False, ci_level="cis")

# uncomment to print

#print(H2_PF.H_PF)



Start SCF iterations:

Canonical RHF One-electron energy = -13.3346745238277062
CQED-RHF One-electron energy      = -13.3338879268738477
Nuclear repulsion energy          = 1.5875316320100001
Dipole energy                     = 0.0004020117084399
SCF Iteration   1: Energy = -7.7668598404831943   dE = -7.76686E+00   dRMS = 6.48590E-05
SCF Iteration   2: Energy = -7.7668600544265214   dE = -2.13943E-07   dRMS = 1.13708E-05
SCF Iteration   3: Energy = -7.7668600673978645   dE = -1.29713E-08   dRMS = 4.90503E-06
SCF Iteration   4: Energy = -7.7668600701681267   dE = -2.77026E-09   dRMS = 2.39509E-06
SCF Iteration   5: Energy = -7.7668600708436557   dE = -6.75529E-10   dRMS = 1.19367E-06
SCF Iteration   6: Energy = -7.7668600710151789   dE = -1.71523E-10   dRMS = 6.05528E-07
SCF Iteration   7: Energy = -7.7668600710603908   dE = -4.52118E-11   dRMS = 3.12866E-07
SCF Iteration   8: Energy = -7.7668600710727418   dE = -1.23510E-11   dRMS = 1.64534E-07
SCF Iteration   9: Energy = -7.766860071

We will diagonalize the Hamiltonian and grab the 
CIS vector for the first singlet excited state of LiH, which turns out to be the
5th root of the CIS Hamiltonian.

In [10]:
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

e_5 = e_fci[4]
c_5 = wavefunctions[:,4]

# check to make sure these vectors are normalized and that the expectation value of the 
# <c_5 | H_PF | c_5> = E_5
E_exp = np.dot(c_5, np.dot( H2_PF.H_PF , c_5 ))

norm = np.dot( c_5 , c_5 )
print("E_5         :", e_5)
print("<E> with c_5:", E_exp)
print("c_5 . c_5   :", norm)

E_5         : -7.5928861957662885
<E> with c_5: -7.592886195766283
c_5 . c_5   : 0.9999999999999998


Now we will print the coefficients associated with each determinant in the determinant list.
Note that the ordering of these determinants goes as follows:
\begin{align}
|R\rangle|0\rangle \\
|S\rangle|0\rangle \\
|R\rangle|1\rangle \\
|S\rangle|1\rangle
\end{align}
where $|S\rangle$ denotes all of the singly-excited determinants.
We will get the total number of electronic determinants ($|R\rangle \& |S\rangle$),
the total number of singly-excited determinants ($|S\rangle$), and the total
number of electronic-photonic determinants 
($|R\rangle|0\rangle \& |S\rangle|0\rangle \& |R\rangle|1\rangle \& |S\rangle|1\rangle = 2(|R\rangle \& |S\rangle)$)

In [11]:
# number of electronic determinants
n_el_dets = len(H2_PF.dets)
# number of singly-excited electronic determinants
n_el_sings = n_el_dets - 1
# number of electronic-photonic determinants
n_elp_dets = len(c_5)

print(n_el_dets)
assert n_elp_dets == 2 * n_el_dets
assert n_el_sings == n_el_dets - 1

17


We can print each determinant along with its corresponding CIS coefficient:

In [12]:
cnt = 0
for i in range(n_el_dets):
    print(F" {c_5[i]:12.4e} : {H2_PF.dets[i]}|0>    {cnt}")
    cnt += 1
    
    
for i in range(n_el_dets):
    print(F" {c_5[i+n_el_dets]:12.4e} : {H2_PF.dets[i]}|1>     {cnt}")
    cnt += 1

    
print("test")
print(H2_PF.dets[2])
a, b = H2_PF.dets[2].getOrbitalIndexLists()
print(a)
print(b)




  -6.4306e-17 : |[0, 1][0, 1]>|0>    0
  -4.1955e-05 : |[1, 2][0, 1]>|0>    1
  -1.2179e-14 : |[1, 3][0, 1]>|0>    2
  -2.1490e-03 : |[1, 4][0, 1]>|0>    3
  -3.5792e-05 : |[1, 5][0, 1]>|0>    4
   7.0723e-03 : |[0, 2][0, 1]>|0>    5
   3.8531e-12 : |[0, 3][0, 1]>|0>    6
   7.0259e-01 : |[0, 4][0, 1]>|0>    7
  -1.3946e-03 : |[0, 5][0, 1]>|0>    8
   4.1955e-05 : |[0, 1][1, 2]>|0>    9
   1.1792e-14 : |[0, 1][1, 3]>|0>    10
   2.1490e-03 : |[0, 1][1, 4]>|0>    11
   3.5792e-05 : |[0, 1][1, 5]>|0>    12
  -7.0723e-03 : |[0, 1][0, 2]>|0>    13
  -3.8326e-12 : |[0, 1][0, 3]>|0>    14
  -7.0259e-01 : |[0, 1][0, 4]>|0>    15
   1.3946e-03 : |[0, 1][0, 5]>|0>    16
   2.4758e-14 : |[0, 1][0, 1]>|1>     17
   5.2127e-04 : |[1, 2][0, 1]>|1>     18
  -4.1876e-16 : |[1, 3][0, 1]>|1>     19
   1.8586e-05 : |[1, 4][0, 1]>|1>     20
   3.9621e-04 : |[1, 5][0, 1]>|1>     21
  -7.0538e-02 : |[0, 2][0, 1]>|1>     22
   1.9093e-13 : |[0, 3][0, 1]>|1>     23
   3.3662e-02 : |[0, 4][0, 1]>|1>     24
  

In [13]:
def spin_idx_to_spat_idx_and_spin(P):
    """ function to take the numeric label of a spin orbital
        and return the spatial index and the spin index separately.
        Starts counting from 0:
        
        Arguments
        ---------
        P : int
            spin orbital label
        
        Returns
        -------
        [p, spin] : numpy array of ints
            p is the spatial orbital index and spin is the spin index.
            spin = 1  -> alpha
            spin = -1 -> beta
            
        Example
        -------
        >>> spin_idx_to_spat_idx_and_spin(0)
        >>> [0, 1]
        >>> spin_idx_to_spat_idx_and_spin(3)
        >>> [1, -1]
        
    """
    spin = 1
    if P % 2 == 0:
        p = P / 2
        spin = 1
    else:
        p = (P-1) / 2
        spin = -1
    return np.array([p, spin], dtype=int)

def calc1RDMElement( C, p_so, q_so, bra_idx, ket_idx):
    """
    calculate c_1^* c_2 <bra|p^{\dagger}q|ket>

    Argument
    --------
    C : numpy array
        the CIS vector 
    p : int 
        spin orbital index for creation operator
    q : int
        spin orbital index for annihilation operator
    bra_idx : int
        index of bra determinant
    ket_idx : int
        index of ket determinant
    """
    
    n_el_dets = len(H2_PF.dets)

    # get coefficients for the appropriate bra/ket states
    C0_bra = C[bra_idx]
    C1_bra = C[bra_idx + n_el_dets]

    C0_ket = C[ket_idx]
    C1_ket = C[ket_idx + n_el_dets]
    
    weight = np.conj(C0_bra) * C0_ket + np.conj(C1_bra) * C1_ket
    elem = 0
    
    # get spatial and spin indices from SO index
    p_map = spin_idx_to_spat_idx_and_spin(p_so)
    q_map = spin_idx_to_spat_idx_and_spin(q_so)
    
    p = p_map[0]
    q = q_map[0]
    
    p_spin = p_map[1]
    q_spin = q_map[1]
    
    
    # see if this is an p, q both belong to alpha or both belong to beta
    if p_spin==1 and q_spin==1:
        block = "alpha"
        #print(F'\n {weight} <{H2_PF.dets[bra_idx]}|{p}a^+ {q}a|{H2_PF.dets[ket_idx]}> \n')
    elif p_spin==-1 and q_spin==-1:
        block = "beta"
        #print(F'\n {weight} <{H2_PF.dets[bra_idx]}|{p}b^+ {q}b|{H2_PF.dets[ket_idx]}> \n')
    # if p and q have different spin, the element is zero
    else:
        block = "mixed"
        

    # alpha block
    if block=="alpha":
        
        # make copy of the ket determinant
        ket_copy = H2_PF.dets[ket_idx].copy()
        
        # assume the element is non-zero
        nonZero=True
        a, b = ket_copy.getOrbitalIndexLists()
        # check to see if q is unoccupied in the alpha list
        if (q not in a):
            nonZero = False

        else:
            # kill q
            ket_copy.removeAlphaOrbital(q)
            #print("removed alpha orbital", q)
            #print("ket_copy",ket_copy)
            #print("original",H2_PF.dets[ket_idx])
            # get new occupation list
            a, b = ket_copy.getOrbitalIndexLists()

        # check to see if p is occupied in the alpha list
        if (p in a):
            nonZero = False

        else:
            # create p
            ket_copy.addAlphaOrbital(p)
            #print("added alpha orbital", p)
            #print("ket_copy",ket_copy)
            #print("original",H2_PF.dets[ket_idx])
        numUniqueOrbitals = H2_PF.dets[bra_idx].numberOfTotalDiffOrbitals(ket_copy)
        if numUniqueOrbitals==0 and nonZero:
            #dum = np.conj(C0_bra) * C0_ket + np.conj(C1_bra) * C1_ket
            #print(F"adding {dum}")
            elem += weight
            
        # alpha block
    elif block=="beta":
        
        # make copy of the ket determinant
        ket_copy = H2_PF.dets[ket_idx].copy()
        
        # assume the element is non-zero
        nonZero=True
        a, b = ket_copy.getOrbitalIndexLists()
        # check to see if q is unoccupied in the alpha list
        if (q not in b):
            nonZero = False

        else:
            # kill q
            ket_copy.removeBetaOrbital(q)
            #print("removed beta orbital", q)
            #print("ket_copy",ket_copy)
            #print("original",H2_PF.dets[ket_idx])
            # get new occupation list
            a, b = ket_copy.getOrbitalIndexLists()

        # check to see if p is occupied in the alpha list
        if (p in b):
            nonZero = False

        else:
            # create p
            ket_copy.addBetaOrbital(p)
            #print("added beta orbital", p)
            #print("ket_copy",ket_copy)
            #print("original",H2_PF.dets[ket_idx])
        numUniqueOrbitals = H2_PF.dets[bra_idx].numberOfTotalDiffOrbitals(ket_copy)
        if numUniqueOrbitals==0 and nonZero:
            #dum = np.conj(C0_bra) * C0_ket + np.conj(C1_bra) * C1_ket
            #print(F"adding {dum}")
            elem += weight
            
    else:
        elem = 0.0
        #print("mixed")
            
    return elem
            







In [14]:
nmo = H2_PF.nmo
nso = 2 * nmo
D = np.zeros((nso, nso))

for p in range(nso):
    for q in range(nso):
        for i in range(n_el_dets):
            for j in range(n_el_dets):
                elem = calc1RDMElement(c_5, p, q, i, j)
                #print(F' {p},{q},{i}, {j}, {elem}')
                D[p, q] += elem


print(D)
print(np.trace(D))
#print(Dbeta)


[[ 0.99999  0.      -0.00154  0.       0.       0.      -0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.99999  0.      -0.00154  0.      -0.       0.       0.       0.      -0.       0.      -0.     ]
 [-0.00154  0.       0.50001  0.      -0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.      -0.00154  0.       0.50001  0.       0.       0.      -0.       0.      -0.       0.      -0.     ]
 [ 0.       0.      -0.       0.       0.00503  0.       0.       0.       0.00259  0.      -0.00101  0.     ]
 [ 0.      -0.       0.       0.       0.       0.00503  0.       0.       0.       0.00259  0.      -0.00101]
 [-0.       0.       0.       0.       0.       0.       0.       0.       0.       0.      -0.       0.     ]
 [ 0.       0.       0.      -0.       0.       0.       0.       0.       0.       0.       0.      -0.     ]
 [ 0.       0.       0.       0.       0.00259  0.       0.       0.       0.49477  0.      -0.0005   0.     ]
 

In [ ]:
#zero = False
#print(H2_PF.dets[2])
#if (q in a):
#    H2_PF.dets[2].removeAlphaOrbital(q)
#    print("removed orbital", q)
#    print(H2_PF.dets[2])
#else:
#    zero = True

#a, b = H2_PF.dets[2].getOrbitalIndexLists()
#if (p not in a):
#    H2_PF.dets[2].addAlphaOrbital(p)
#    print("added orbital", p)
#    print(H2_PF.dets[2])
#else:
#    zero = True
#H2_PF.dets[2].removeBetaOrbital(0)
#print(zero, H2_PF.dets[2])

It is more helpful to associate the excitation indices with each determinant so that the 
corresponding CIS coefficient can be associated with its orbital labels.  
For example:

\begin{align}
-6.3453e-03 &: |1, 2,\overline{0},\overline{1}\rangle|0\rangle \rightarrow c^0_{0,2} \\
-6.3453e-03 &: |0, 1,\overline{1},\overline{2}\rangle|0\rangle \rightarrow c^0_{\overline{0},\overline{2}} \\
0.0000e+00  &: |0, 5,\overline{0},\overline{1}\rangle|1\rangle \rightarrow c^1_{1,5}
\end{align}

The following block will use the methods of the Determinant class to make this correspondence.
We will form 4 different arrays each with size $nmo x nmo$.  

`c0_alpha[i,a]` : $c^0_{i,a}$

`c0_beta[i,a]` : $c^0_{\overline{i},\overline{a}}$

`c1_alpha[i,a]` : $c^1_{i,a}$

`c1_beta[i,a]` : $c^1_{\overline{i},\overline{a}}$

to hold these coefficients.


In [ ]:
# initialize the 4 arrays that will store the coefficients
nmo = H2_PF.nmo
c0_alpha = np.zeros((nmo,nmo))
c1_alpha = np.zeros_like(c0_alpha)
c0_beta = np.zeros_like(c0_alpha)
c1_beta = np.zeros_like(c0_alpha)


# get the occupation string to use for the reference
occList = [i for i in range(H2_PF.ndocc)]

print("Reference occupation string is", occList)

# generate the reference determinant using this occupation string
# for both alpha and beta spin orbitals
det_ref = Determinant(alphaObtList=occList, betaObtList=occList)
print("Reference Determinant:", det_ref)

# get a list of occupied alpha and beta spin orbitals
alphaO, betaO = det_ref.getOrbitalIndexLists()
print("Occupied Alpha Orbital List", alphaO)
print("Occupied Beta Orbital List ", betaO)

# get a list of unnoccupied alpha and beta spin orbitals
alphaU, betaU = det_ref.getUnoccupiedOrbitalsInLists(H2_PF.nmo)
print("Unoccupied Alpha Orbital List", alphaU)
print("Unoccupied Beta Orbital List ", betaU)

# loop through the i, a pairs for the alpha orbitals and
# map the coefficients to nmoxnmo c arrays
_ci = 1
for i in alphaO:
    for a in alphaU:
        c0_alpha[i,a] = c_5[_ci]
        c1_alpha[i,a] = c_5[_ci + n_el_dets]
        _ci += 1
        

for i in betaO:
    for a in betaU:
        c0_beta[i, a] = c_5[_ci]
        c1_beta[i, a] = c_5[_ci + n_el_dets]
        _ci += 1

print(c0_alpha)
print(c0_beta)
print(c1_alpha)
print(c1_beta)

In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

#print the ground-state and first excited state
#print("Psi4 |G> is ",fci_e)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])
#|G> is  -75.01298018271805
#|LP> is  -75.01298018271805
assert np.isclose(e_fci[0],-74.94207989868084)

In [ ]:
cqed_cis_dict = cs_cqed_cis(lambda_vector, omega_val, mol_str, options_dict)
_c_vecs = cqed_cis_dict["CQED-CIS L VECTORS"]



In [ ]:
c_0 = _c_vecs[:,0]
c_1 = _c_vecs[:,1]
c_2 = _c_vecs[:,2]
c_3 = _c_vecs[:,3]
c_4 = _c_vecs[:,4]

print(c_4)

In [ ]:
_nmo = H2_PF.nmo
_ndocc = H2_PF.ndocc
_nvirt = _nmo - _ndocc

# total number of states in {|R,0>, |R,1>, |S,0>, |S,1>}
_n_ss = len(c_1)
# total number of states in {|R,0>, |S,0>} and in {|R,1>, |S,1>}
_n_s = int(_n_ss / 2)

# right now the CI configurations are odered like the following:
# |R,0>, |R,1>, |S1,0>, |S1,1>, |S2,0>, |S2,1>, ...
# where |S1> denotes the first singly-excited derminant, |S2> the second, etc
# so the configurations with 0 photon occupation have even indices and
# the configurations with 1 photon occupation have odd indices.

_pvac_idx = np.arange(0, _n_ss, 2)  # <== indices for C coeffs for |0> states
_pone_idx = np.arange(1, _n_ss, 2)  # <== indices for C coeffs for |1> states

# CIS coefficients spanning the |R,0> and |S,0> states
c_n0 = c_1[_pvac_idx]
# CIS coefficients spanning the |R,1> and |R,1> states
c_n1 = c_1[_pone_idx]

# now each vector spans the |R> + {|S>} basis for a given photon state.
# for the {|S>}, let's generate a list of the excitations in the order
# of the |S> basis
_pi = np.zeros(_nmo)
_xi = np.zeros(_nmo)


excitations = []
C0_ia = np.zeros((_nmo, _nmo))
C1_ia = np.zeros((_nmo, _nmo))
_idx = 1
for _i in range(_ndocc):
    _xi[_i] = 1
    for _a in range(_nvirt):
        _A = _a + _ndocc
        _pi[_A] = 1
        excitations.append((_i, _A))
        C0_ia[_i, _A] = c_n0[_idx]
        C1_ia[_i, _A] = c_n1[_idx]
        _idx += 1

print(c_n0)
print(c_n1)
print(np.dot(c_n0, c_n0))
print(np.dot(c_n1, c_n1))

print(_pi)

In [ ]:


_D1 = np.zeros((_nmo, _nmo))
for _i in range(_ndocc):
    _D1[_i, _i] = c_n0[0] * c_n0[0] + c_n1[0] * c_n1[0]

for _p in range(_nmo):
    for _q in range(_nmo):
        _D1[_p, _q] += c_n0[0] * C0_ia[_p, _q] * _pi[_p] * _xi[_q]
        _D1[_p, _q] += c_n0[0] * C0_ia[_p, _q] * _xi[_p] * _pi[_q]
        _D1[_p, _q] += c_n1[0] * C1_ia[_p, _q] * _pi[_p] * _xi[_q]
        _D1[_p, _q] += c_n1[0] * C1_ia[_p, _q] * _xi[_p] * _pi[_q]

        for _i in range(_ndocc):
            _D1[_p, _q] += C0_ia[_p, _i] * C0_ia[_i, _q] * _pi[_p] * _pi[_q]
            _D1[_p, _q] += C1_ia[_p, _i] * C1_ia[_i, _q] * _pi[_p] * _pi[_q]

        for _a in range(_nvirt):
            _A = _a + _ndocc
            _D1[_p, _q] -= C0_ia[_a, _q] * C0_ia[_p, _a] * _xi[_p] * _xi[_q]
            _D1[_p, _q] -= C1_ia[_a, _q] * C1_ia[_p, _a] * _xi[_p] * _xi[_q]


ndocc = 5
n_act_el = 4
n_act_orb = 4

n_in_orb=ndocc-n_act_el//2
n_ac_el_half=n_act_el//2
print(n_ac_el_half)
inactive_list=list(x for x in range(n_in_orb))
print(inactive_list)
print('Generating all determinants in active space')
t = time.time()
singlesDets = []
for alpha in combinations(range(n_act_orb), n_ac_el_half):
    alphalist=list(alpha)
    alphalist=[x+n_in_orb for x in alphalist]
    alphalist[0:0]=inactive_list
    alpha=tuple(alphalist)
    #alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(n_act_orb), n_ac_el_half):
        betalist=list(beta)
        betalist=[x+n_in_orb for x in betalist]
        betalist[0:0]=inactive_list
        beta=tuple(betalist)
        #beta_ex_level = compute_excitation_level(beta, ndocc)
        #if alpha_ex_level + beta_ex_level == 1:
        print(F' adding alpha: {alpha} and beta: {beta}\n') 
        singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))